In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Daily Chart-Pattern Scanner (schedule-friendly) + Plot Export

Scans the latest daily bar for breakout confirmations of BULLISH patterns
(Tier-1 + selected Tier-2) and prints a concise line per hit:

  YYYY-MM-DD  TICKER  LONG  <PatternName>  <reason>

Also saves a PNG chart per hit with key levels annotated.

Excluded: candlestick set (as requested).

Patterns implemented:
  Tier-1: Double Bottom, Ascending Triangle, Bullish Rectangle, Symmetrical Triangle (up-break)
  Tier-2: Inverse H&S, Bullish Flag/Pennant, Falling Wedge, Cup & Handle, Ascending Channel

Rules/filters:
  • Trend filter: Close > MA200 AND MA200 slope > 0
  • Volume confirmation: Volume > vol_confirm_mult × VOL_MA20
  • Breakout must occur on the most recent bar

Inputs (universe):
  • Env var SCAN_TICKERS="RELIANCE.NS,TCS.NS,HDFCBANK.NS" (comma/space sep), or
  • File ./tickers.txt (one symbol per line), or
  • Default list in DEFAULT_TICKERS below

Outputs:
  • Prints matches to stdout
  • Writes CSV to ./signals/signals_YYYY-MM-DD.csv
  • Saves pattern plots to ./signals/plots/<ticker>_<date>_<pattern>.png when SCAN_SAVE_PLOTS=1

Scheduling (example, run Mon–Fri 18:30 IST after market close):
  $ crontab -e
  TZ=Asia/Kolkata
  30 18 * * 1-5 /usr/bin/python3 /path/to/daily_pattern_scanner.py >> /path/to/scan.log 2>&1

Dependencies:
  pip install yfinance pandas numpy matplotlib
"""

import os, sys, math, warnings, logging
from dataclasses import dataclass
from typing import List, Optional, Tuple, Dict, Any

import numpy as np
import pandas as pd

try:
    import yfinance as yf
except Exception:
    yf = None

# matplotlib + mplfinance for plot export
import matplotlib
matplotlib.use("Agg")  # for headless servers
import matplotlib.pyplot as plt
try:
    import mplfinance as mpf
except Exception:
    mpf = None
import matplotlib.dates as mdates
try:
    from mplfinance.original_flavor import candlestick_ohlc
except Exception:
    candlestick_ohlc = None


warnings.filterwarnings("ignore", category=FutureWarning)

# =========================
# CONFIG (tune here)
# =========================
DEFAULT_TICKERS = ['360ONE.NS', '3MINDIA.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABB.NS', 'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABLBL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'ACC.NS', 'ACE.NS', 'ACMESOLAR.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'AEGISLOG.NS', 'AEGISVOPAK.NS', 'AFCONS.NS', 'AFFLE.NS', 'AGARWALEYE.NS', 'AIAENG.NS', 'AIIL.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'AKZOINDIA.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APLAPOLLO.NS', 'APLLTD.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ARE&M.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAL.NS', 'ASTRAZEN.NS', 'ATGL.NS', 'ATHERENERG.NS', 'ATUL.NS', 'AUBANK.NS', 'AUROPHARMA.NS', 'AWL.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJAJFINSV.NS', 'BAJAJHFL.NS', 'BAJAJHLDNG.NS', 'BAJFINANCE.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'BASF.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BBTC.NS', 'BDL.NS', 'BEL.NS', 'BEML.NS', 'BERGEPAINT.NS', 'BHARATFORG.NS', 'BHARTIARTL.NS', 'BHARTIHEXA.NS', 'BHEL.NS', 'BIKAJI.NS', 'BIOCON.NS', 'BLS.NS', 'BLUEDART.NS', 'BLUEJET.NS', 'BLUESTARCO.NS', 'BOSCHLTD.NS', 'BPCL.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'BSE.NS', 'BSOFT.NS', 'CAMPUS.NS', 'CAMS.NS', 'CANBK.NS', 'CANFINHOME.NS', 'CAPLIPOINT.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CCL.NS', 'CDSL.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CENTURYPLY.NS', 'CERA.NS', 'CESC.NS', 'CGCL.NS', 'CGPOWER.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOICEIN.NS', 'CHOLAFIN.NS', 'CHOLAHLDNG.NS', 'CIPLA.NS', 'CLEAN.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COHANCE.NS', 'COLPAL.NS', 'CONCOR.NS', 'CONCORDBIO.NS', 'COROMANDEL.NS', 'CRAFTSMAN.NS', 'CREDITACC.NS', 'CRISIL.NS', 'CROMPTON.NS', 'CUB.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAPATTNS.NS', 'DBREALTY.NS', 'DCMSHRIRAM.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELHIVERY.NS', 'DEVYANI.NS', 'DIVISLAB.NS', 'DIXON.NS', 'DLF.NS', 'DMART.NS', 'DOMS.NS', 'DRREDDY.NS', 'ECLERX.NS', 'EICHERMOT.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'ELECON.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'EMCURE.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ENRIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'ETERNAL.NS', 'EXIDEIND.NS', 'FACT.NS', 'FEDERALBNK.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FIRSTCRY.NS', 'FIVESTAR.NS', 'FLUOROCHEM.NS', 'FORCEMOT.NS', 'FORTIS.NS', 'FSL.NS', 'GAIL.NS', 'GESHIP.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'GMDCLTD.NS', 'GMRAIRPORT.NS', 'GODFRYPHLP.NS', 'GODIGIT.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GPIL.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GRSE.NS', 'GSPL.NS', 'GUJGASLTD.NS', 'GVT&D.NS', 'HAL.NS', 'HAPPSTMNDS.NS', 'HAVELLS.NS', 'HBLENGINE.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEG.NS', 'HEROMOTOCO.NS', 'HEXT.NS', 'HFCL.NS', 'HINDALCO.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'HOMEFIRST.NS', 'HONASA.NS', 'HONAUT.NS', 'HSCL.NS', 'HUDCO.NS', 'HYUNDAI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDBI.NS', 'IDEA.NS', 'IDFCFIRSTB.NS', 'IEX.NS', 'IFCI.NS', 'IGIL.NS', 'IGL.NS', 'IIFL.NS', 'IKS.NS', 'INDGN.NS', 'INDHOTEL.NS', 'INDIACEM.NS', 'INDIAMART.NS', 'INDIANB.NS', 'INDIGO.NS', 'INDUSINDBK.NS', 'INDUSTOWER.NS', 'INFY.NS', 'INOXINDIA.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'IOB.NS', 'IOC.NS', 'IPCALAB.NS', 'IRB.NS', 'IRCON.NS', 'IRCTC.NS', 'IREDA.NS', 'IRFC.NS', 'ITC.NS', 'ITCHOTELS.NS', 'ITI.NS', 'J&KBANK.NS', 'JBCHEPHARM.NS', 'JBMA.NS', 'JINDALSAW.NS', 'JINDALSTEL.NS', 'JIOFIN.NS', 'JKCEMENT.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JPPOWER.NS', 'JSL.NS', 'JSWENERGY.NS', 'JSWINFRA.NS', 'JSWSTEEL.NS', 'JUBLFOOD.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JWL.NS', 'JYOTHYLAB.NS', 'JYOTICNC.NS', 'KAJARIACER.NS', 'KALYANKJIL.NS', 'KARURVYSYA.NS', 'KAYNES.NS', 'KEC.NS', 'KEI.NS', 'KFINTECH.NS', 'KIMS.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KOTAKBANK.NS', 'KPIL.NS', 'KPITTECH.NS', 'KPRMILL.NS', 'KSB.NS', 'LALPATHLAB.NS', 'LATENTVIEW.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICHSGFIN.NS', 'LICI.NS', 'LINDEINDIA.NS', 'LLOYDSME.NS', 'LODHA.NS', 'LT.NS', 'LTF.NS', 'LTFOODS.NS', 'LTIM.NS', 'LTTS.NS', 'LUPIN.NS', 'M&M.NS', 'M&MFIN.NS', 'MAHABANK.NS', 'MAHSCOOTER.NS', 'MAHSEAMLES.NS', 'MANAPPURAM.NS', 'MANKIND.NS', 'MANYAVAR.NS', 'MAPMYINDIA.NS', 'MARICO.NS', 'MARUTI.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MCX.NS', 'MEDANTA.NS', 'METROPOLIS.NS', 'MFSL.NS', 'MGL.NS', 'MINDACORP.NS', 'MMTC.NS', 'MOTHERSON.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MRF.NS', 'MRPL.NS', 'MSUMI.NS', 'MUTHOOTFIN.NS', 'NAM-INDIA.NS', 'NATCOPHARM.NS', 'NATIONALUM.NS', 'NAUKRI.NS', 'NAVA.NS', 'NAVINFLUOR.NS', 'NBCC.NS', 'NCC.NS', 'NESTLEIND.NS', 'NETWEB.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NH.NS', 'NHPC.NS', 'NIACL.NS', 'NIVABUPA.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NSLNISP.NS', 'NTPC.NS', 'NTPCGREEN.NS', 'NUVAMA.NS', 'NUVOCO.NS', 'NYKAA.NS', 'OBEROIRLTY.NS', 'OFSS.NS', 'OIL.NS', 'OLAELEC.NS', 'OLECTRA.NS', 'ONESOURCE.NS', 'ONGC.NS', 'PAGEIND.NS', 'PATANJALI.NS', 'PAYTM.NS', 'PCBL.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFC.NS', 'PFIZER.NS', 'PGEL.NS', 'PGHH.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PIIND.NS', 'PNB.NS', 'PNBHOUSING.NS', 'POLICYBZR.NS', 'POLYCAB.NS', 'POLYMED.NS', 'POONAWALLA.NS', 'POWERGRID.NS', 'POWERINDIA.NS', 'PPLPHARMA.NS', 'PRAJIND.NS', 'PREMIERENE.NS', 'PRESTIGE.NS', 'PTCIL.NS', 'PVRINOX.NS', 'RADICO.NS', 'RAILTEL.NS', 'RAINBOW.NS', 'RAMCOCEM.NS', 'RBLBANK.NS', 'RCF.NS', 'RECLTD.NS', 'REDINGTON.NS', 'RELIANCE.NS', 'RELINFRA.NS', 'RHIM.NS', 'RITES.NS', 'RKFORGE.NS', 'RPOWER.NS', 'RRKABEL.NS', 'RVNL.NS', 'SAGILITY.NS', 'SAIL.NS', 'SAILIFE.NS', 'SAMMAANCAP.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SBFC.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SBIN.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SHREECEM.NS', 'SHRIRAMFIN.NS', 'SHYAMMETL.NS', 'SIEMENS.NS', 'SIGNATURE.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONACOMS.NS', 'SONATSOFTW.NS', 'SRF.NS', 'STARHEALTH.NS', 'SUMICHEM.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUNPHARMA.NS', 'SUNTV.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'SWANCORP.NS', 'SWIGGY.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TARIL.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATATECH.NS', 'TBOTEK.NS', 'TCS.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'THELEELA.NS', 'THERMAX.NS', 'TIINDIA.NS', 'TIMKEN.NS', 'TITAGARH.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRITURBINE.NS', 'TRIVENI.NS', 'TTML.NS', 'TVSMOTOR.NS', 'UBL.NS', 'UCOBANK.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UNITDSPR.NS', 'UNOMINDA.NS', 'UPL.NS', 'USHAMART.NS', 'UTIAMC.NS', 'VBL.NS', 'VEDL.NS', 'VENTIVE.NS', 'VGUARD.NS', 'VIJAYA.NS', 'VMM.NS', 'VOLTAS.NS', 'VTL.NS', 'WAAREEENER.NS', 'WELCORP.NS', 'WELSPUNLIV.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZEEL.NS', 'ZENSARTECH.NS', 'ZENTEC.NS', 'ZFCVINDIA.NS', 'ZYDUSLIFE.NS']
START_DATE = os.environ.get("SCAN_START", "2015-01-01")
END_DATE = None  # None = up to latest

# Detection knobs
ZIGZAG_PCT = float(os.environ.get("ZIGZAG_PCT", 4.0))        # 3–6 typical
VOL_CONFIRM_MULT = float(os.environ.get("VOL_CONFIRM_MULT", 1.2))  # 1.1–1.3 typical
FLAT_STDEV_THRESH = float(os.environ.get("FLAT_STDEV_THRESH", 0.015))  # ~1–2%
SHOULDER_TOL = float(os.environ.get("SHOULDER_TOL", 0.08))   # 5–8%
MAX_FLAG_LEN = int(os.environ.get("MAX_FLAG_LEN", 20))
MIN_CUP_BARS = int(os.environ.get("MIN_CUP_BARS", 30))
MAX_HANDLE_DEPTH = float(os.environ.get("MAX_HANDLE_DEPTH", 0.35))

# Trend filter params
MA_SHORT = 50
MA_LONG = 200
MA_LONG_SLOPE_LOOKBACK = 5  # MA200[t] > MA200[t-5]

# Output dirs & plotting
OUT_DIR = os.environ.get("SCAN_OUT_DIR", "signals")
SAVE_PLOTS = os.environ.get("SCAN_SAVE_PLOTS", "1").strip() not in ("0", "false", "False", "no")
PLOT_DIR = os.path.join(OUT_DIR, "plots")
PLOT_BARS = int(os.environ.get("SCAN_PLOT_BARS", 240))
PLOT_PAD_DAYS = int(os.environ.get("SCAN_PLOT_PAD_DAYS", 7))  # right-side padding (axis extension) to avoid clipping

# =========================
# LOGGING
# =========================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
log = logging.getLogger("daily_pattern_scanner")

# =========================
# UTILITIES
# =========================

def ensure_dir(path: str):
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)


def parse_tickers() -> List[str]:
    env = os.environ.get("SCAN_TICKERS", "").strip()
    if env:
        if "," in env:
            toks = [t.strip() for t in env.replace(";", ",").split(",") if t.strip()]
        else:
            toks = [t.strip() for t in env.split() if t.strip()]
        if toks:
            return toks
    if os.path.exists("tickers.txt"):
        with open("tickers.txt", "r", encoding="utf-8") as f:
            lines = [ln.strip() for ln in f if ln.strip() and not ln.startswith("#")]
        if lines:
            return lines
    return DEFAULT_TICKERS


def rolling_slope(x: pd.Series, lookback: int = MA_LONG_SLOPE_LOOKBACK) -> pd.Series:
    return x / x.shift(lookback) - 1.0


def flat_enough(series: pd.Series, thresh: float) -> bool:
    s = series.dropna()
    if len(s) < 5:
        return False
    mu = s.mean()
    if mu == 0:
        return False
    return (s.std() / abs(mu)) < thresh


def vol_confirm_row(row: pd.Series) -> bool:
    return row["Volume"] > VOL_CONFIRM_MULT * max(1e-9, row["VOL_MA20"])


def trend_filter_row(row: pd.Series) -> bool:
    return (row["Close"] > row["MA200"]) and (row["MA200_slope"] > 0)

# =========================
# DATA
# =========================

def load_ohlcv(ticker: str, start: str, end: Optional[str] = None) -> pd.DataFrame:
    if yf is None:
        raise RuntimeError("yfinance is not available. pip install yfinance")
    df = yf.download(ticker, start=start, end=end, auto_adjust=False, progress=False, multi_level_index=False)
    df = df.dropna()
    if df.empty:
        return df
    # Indicators
    df["MA50"] = df["Close"].rolling(MA_SHORT).mean()
    df["MA200"] = df["Close"].rolling(MA_LONG).mean()
    df["MA200_slope"] = rolling_slope(df["MA200"], MA_LONG_SLOPE_LOOKBACK)
    df["VOL_MA20"] = df["Volume"].rolling(20).mean()
    return df

# =========================
# ZIGZAG (percent swings)
# =========================

def zigzag_indices(close: pd.Series, pct_threshold: float) -> List[pd.Timestamp]:
    p = pct_threshold / 100.0
    idx = close.index
    piv = [idx[0]]
    last_px = close.iloc[0]
    last_dir = 0  # +1 up, -1 down
    for i in range(1, len(close)):
        chg = (close.iloc[i] - last_px) / last_px
        if last_dir >= 0 and chg >= p:
            last_dir = +1; last_px = close.iloc[i]; piv.append(idx[i])
        elif last_dir <= 0 and chg <= -p:
            last_dir = -1; last_px = close.iloc[i]; piv.append(idx[i])
        else:
            if last_dir == +1 and close.iloc[i] > last_px:
                last_px = close.iloc[i]; piv[-1] = idx[i]
            elif last_dir == -1 and close.iloc[i] < last_px:
                last_px = close.iloc[i]; piv[-1] = idx[i]
    return sorted(set(piv))


def recent_swings(df: pd.DataFrame, n_last: int = 12) -> pd.DataFrame:
    zz = zigzag_indices(df["Close"], ZIGZAG_PCT)
    return df.loc[zz].tail(n_last)

# =========================
# PLOT UTIL
# =========================

def save_pattern_plot(ticker: str, df: pd.DataFrame, as_of: pd.Timestamp,
                      pattern: str, reason: str, plot_info: Dict[str, Any],
                      out_dir: str = PLOT_DIR, bars: int = PLOT_BARS) -> str:
    """
    Robust candlestick plot with pattern overlays.
    - Tries mplfinance; if it fails to render candles, falls back to Matplotlib candlesticks.
    - No MAs/EMAs. Prevents right-edge clipping by extending x-limits.
    """
    ensure_dir(out_dir)

    w = df.loc[:as_of].tail(bars).copy()
    if w.empty:
        return ""
    cols = [c for c in ["Open", "High", "Low", "Close", "Volume"] if c in w.columns]
    w = w[cols].dropna(subset=["Open", "High", "Low", "Close"])
    if w.empty:
        return ""

    # ---- Build overlays data (we’ll draw them in both engines)
    hlines = [(float(p), str(lbl)) for p, lbl in plot_info.get("hlines", [])]
    segments = [ (pd.to_datetime(x1), float(y1), pd.to_datetime(x2), float(y2))
                 for (x1, y1), (x2, y2) in plot_info.get("segments", []) ]
    points   = [ (str(lbl), pd.to_datetime(x), float(y)) for lbl, x, y in plot_info.get("points", []) ]

    fig, ax = None, None
    used_fallback = False

    # ---- Try mplfinance first
    try:
        h_cfg = dict(hlines=[p for p,_ in hlines], colors='tab:blue', linewidths=1.8, linestyle='--', alpha=0.95) if hlines else None
        a_cfg = dict(alines=[[(x1,y1),(x2,y2)] for x1,y1,x2,y2 in segments], colors='crimson', linewidths=2.0, linestyle='--', alpha=0.95) if segments else None

        fig, axlist = mpf.plot(
            w, type="candle", style="yahoo",
            datetime_format="%Y-%m", xrotation=0, volume=False,
            figsize=(11, 5.5), tight_layout=False, returnfig=True,
            hlines=h_cfg, alines=a_cfg
        )
        ax = axlist[0]

        # Quick sanity check: if no candles landed (rare backend quirk), fall back
        if not ax.collections and not ax.patches:
            used_fallback = True
            plt.close(fig)
    except Exception:
        used_fallback = True

    # ---- Fallback: plain Matplotlib candlesticks
    if used_fallback:
        if candlestick_ohlc is None:
            raise RuntimeError("mplfinance fallback not available. Install: pip install mplfinance")
        fig, ax = plt.subplots(figsize=(11, 5.5))
        d = w.reset_index().rename(columns={w.index.name or "index": "Date"})
        d["DateNum"] = mdates.date2num(pd.to_datetime(d["Date"]))
        ohlc = d[["DateNum", "Open", "High", "Low", "Close"]].values
        candlestick_ohlc(ax, ohlc, width=0.6, colorup="tab:green", colordown="tab:red", alpha=0.9)
        ax.xaxis_date()
        ax.grid(True, linewidth=0.4)

        # overlays
        for price, _lbl in hlines:
            ax.hlines(price, xmin=w.index[0], xmax=w.index[-1], colors='tab:blue', linestyles='--', linewidth=1.8, alpha=0.95)
        for x1,y1,x2,y2 in segments:
            ax.plot([x1, x2], [y1, y2], linestyle='--', linewidth=2.0, color='crimson', alpha=0.95)

    # ---- Labels & layout (both engines)
    # Right padding to avoid clipping
    if PLOT_PAD_DAYS > 0:
        ax.set_xlim(w.index[0], w.index[-1] + pd.Timedelta(days=PLOT_PAD_DAYS))

    # Make sure hlines are visible within y-lims
    if hlines:
        ymin = min(w["Low"].min(), min(p for p,_ in hlines)) * 0.98
        ymax = max(w["High"].max(), max(p for p,_ in hlines)) * 1.02
        ax.set_ylim(ymin, ymax)

    # annotate labels for hlines ~85% into the window
    if hlines:
        x_label = w.index[int(max(1, round(len(w) * 0.85))) - 1]
        for price, label in hlines:
            ax.text(pd.to_datetime(x_label), price, f"{label}:{price:.2f}",
                    va="bottom", ha="left", fontsize=9, color="tab:blue",
                    backgroundcolor="white", alpha=0.7, clip_on=False)

    # points
    for lbl, x, y in points:
        ax.scatter([x], [y], s=28, color="black", zorder=6)
        ax.annotate(lbl, xy=(x, y), xytext=(4, 6), textcoords="offset points", fontsize=9, color="black")

    title = f"{ticker} | {pattern} @ {as_of.date()}  {reason}"
    fig.suptitle(title, fontsize=14, fontweight="bold", y=0.97)
    ax.margins(x=0.02, y=0.10)
    plt.subplots_adjust(top=0.88, right=0.97, left=0.08, bottom=0.12)

    fname = f"{ticker.replace('.', '_')}_{as_of.date()}_{pattern.replace(' ', '_').replace('&','and').replace('/', '')}.png"
    path = os.path.join(out_dir, fname)
    fig.savefig(path, dpi=190)
    plt.close(fig)
    return path

# =========================
# PATTERN DETECTORS (BULLISH)
# Each returns (pattern_name, side, reason_str, plot_info) OR None
# Breakout must be confirmed on the last bar of the window
# =========================

def detect_double_bottom(win: pd.DataFrame) -> Optional[Tuple[str, str, str, Dict[str, Any]]]:
    pts = recent_swings(win, 10)
    if len(pts) < 4:
        return None
    lows = pts["Low"]
    two = lows.nsmallest(2)
    if len(two) < 2:
        return None
    l1i, l2i = two.index[0], two.index[1]
    if abs(win.loc[l1i, "Low"] / win.loc[l2i, "Low"] - 1) > 0.015:
        return None
    neckline = win.loc[min(l1i, l2i):max(l1i, l2i)]["High"].max()
    t = win.index[-1]
    row = win.loc[t]
    if row["Close"] > neckline and vol_confirm_row(row) and trend_filter_row(row):
        plot = {
            "hlines": [(neckline, "Neckline")],
            "points": [("B1", l1i, win.loc[l1i, "Low"]), ("B2", l2i, win.loc[l2i, "Low"])]
        }
        return ("Double Bottom", "long", f"Neckline break {neckline:.2f}", plot)
    return None


def detect_ascending_triangle(win: pd.DataFrame) -> Optional[Tuple[str, str, str, Dict[str, Any]]]:
    w = win.tail(60)
    H, L = w["High"], w["Low"]
    top = H.rolling(5).max().dropna()
    bot = L.rolling(5).min().dropna()
    if len(top) < 10 or len(bot) < 10:
        return None
    t = w.index[-1]
    row = w.loc[t]
    if not (flat_enough(top, FLAT_STDEV_THRESH) and bot.iloc[-1] > bot.iloc[0] * 1.02):
        return None
    brk = top.mean()
    if row["Close"] > brk and vol_confirm_row(row) and trend_filter_row(row):
        # rising lows line (approx): straight line from first to last bot
        x1, x2 = bot.index[0], bot.index[-1]
        y1, y2 = bot.iloc[0], bot.iloc[-1]
        plot = {
            "hlines": [(brk, "Flat Top")],
            "segments": [((x1, y1), (x2, y2))],
        }
        return ("Ascending Triangle", "long", f"Breakout above flat top ~{brk:.2f}", plot)
    return None


def detect_bullish_rectangle(win: pd.DataFrame) -> Optional[Tuple[str, str, str, Dict[str, Any]]]:
    w = win.tail(40)
    H, L = w["High"], w["Low"]
    top_series = H.rolling(10).max().dropna()
    bot_series = L.rolling(10).min().dropna()
    if top_series.empty or bot_series.empty:
        return None
    top = top_series.iloc[-1]
    bot = bot_series.iloc[-1]
    if not (flat_enough(top_series, FLAT_STDEV_THRESH) and flat_enough(bot_series, FLAT_STDEV_THRESH)):
        return None
    t = w.index[-1]
    row = w.loc[t]
    if row["Close"] > top and vol_confirm_row(row) and trend_filter_row(row):
        plot = {"hlines": [(top, "Range High"), (bot, "Range Low")]} 
        return ("Bullish Rectangle", "long", f"Range breakout above {top:.2f} (range low {bot:.2f})", plot)
    return None


def detect_sym_triangle_up(win: pd.DataFrame) -> Optional[Tuple[str, str, str, Dict[str, Any]]]:
    w = win.tail(60)
    H, L = w["High"], w["Low"]
    top = H.rolling(5).max().dropna()
    bot = L.rolling(5).min().dropna()
    if len(top) < 10 or len(bot) < 10:
        return None
    if not (top.iloc[-1] < top.iloc[0] * 0.995 and bot.iloc[-1] > bot.iloc[0] * 1.005):
        return None
    brk = top.iloc[-10:].mean()
    t = w.index[-1]
    row = w.loc[t]
    if row["Close"] > brk and vol_confirm_row(row) and trend_filter_row(row):
        # draw approximate converging lines
        plot = {
            "hlines": [(brk, "Upper TL (~avg)")],
            "segments": [((top.index[0], top.iloc[0]), (top.index[-1], top.iloc[-1])),
                          ((bot.index[0], bot.iloc[0]), (bot.index[-1], bot.iloc[-1]))],
        }
        return ("Symmetrical Triangle (Up)", "long", f"Upper trendline break ~{brk:.2f}", plot)
    return None

# ---- Tier-2 simplified ----

def detect_inverse_hs(win: pd.DataFrame) -> Optional[Tuple[str, str, str, Dict[str, Any]]]:
    pts = recent_swings(win, 12)
    if len(pts) < 5:
        return None
    lows = pts["Low"]
    mid_i = lows.idxmin()  # head
    left = pts.loc[:mid_i]
    right = pts.loc[mid_i:]
    if len(left) < 2 or len(right) < 2:
        return None
    L1i = left["Low"].iloc[:-1].idxmin()
    L3i = right["Low"].iloc[1:].idxmin()
    if abs(win.loc[L1i, "Low"] / win.loc[L3i, "Low"] - 1) > SHOULDER_TOL:
        return None
    nl = np.mean([
        win.loc[min(L1i, mid_i):max(L1i, mid_i)]["High"].max(),
        win.loc[min(mid_i, L3i):max(mid_i, L3i)]["High"].max(),
    ])
    t = win.index[-1]
    row = win.loc[t]
    if row["Close"] > nl and vol_confirm_row(row) and trend_filter_row(row):
        plot = {"hlines": [(nl, "Neckline")],
                "points": [("LS", L1i, win.loc[L1i, "Low"]), ("H", mid_i, win.loc[mid_i, "Low"]), ("RS", L3i, win.loc[L3i, "Low"])]}
        return ("Inverse Head & Shoulders", "long", f"Neckline break {nl:.2f}", plot)
    return None


def detect_flag_pennant(win: pd.DataFrame) -> Optional[Tuple[str, str, str, Dict[str, Any]]]:
    w = win.tail(40)
    if len(w) < 25:
        return None
    ret20 = w["Close"].iloc[-1] / w["Close"].iloc[-21] - 1.0
    atrp = (w["High"] - w["Low"]) / w["Close"]
    impulse = (ret20 > 0.1) and (atrp.rolling(10).mean().iloc[-1] > atrp.rolling(10).mean().iloc[-11])
    if not impulse:
        return None
    cons = w.tail(MAX_FLAG_LEN)
    H, L = cons["High"], cons["Low"]
    top = H.rolling(5).max().dropna()
    bot = L.rolling(5).min().dropna()
    t = cons.index[-1]
    row = cons.loc[t]
    is_flag = (top.iloc[-1] <= top.iloc[0] * 1.01) and (bot.iloc[-1] <= bot.iloc[0] * 1.01)
    is_penn = (top.iloc[-1] < top.iloc[0]) and (bot.iloc[-1] > bot.iloc[0])
    brk = top.mean()
    if (is_flag or is_penn) and row["Close"] > brk and vol_confirm_row(row) and trend_filter_row(row):
        label = "Bullish Flag" if is_flag else "Bullish Pennant"
        plot = {"hlines": [(brk, "Break")],
                "segments": [((top.index[0], top.iloc[0]), (top.index[-1], top.iloc[-1])),
                              ((bot.index[0], bot.iloc[0]), (bot.index[-1], bot.iloc[-1]))]}
        return (label, "long", f"Break above consolidation ~{brk:.2f}", plot)
    return None


def detect_falling_wedge(win: pd.DataFrame) -> Optional[Tuple[str, str, str, Dict[str, Any]]]:
    w = win.tail(60)
    H, L = w["High"], w["Low"]
    down_top = H.iloc[-1] < H.iloc[0] * 0.98
    down_bot = L.iloc[-1] < L.iloc[0] * 0.98
    narrowing = (H.max() - L.min()) > 1.2 * (H.tail(10).max() - L.tail(10).min())
    if not (down_top and down_bot and narrowing):
        return None
    upper_avg = H.rolling(5).max().dropna().mean()
    t = w.index[-1]
    row = w.loc[t]
    if row["Close"] > upper_avg and vol_confirm_row(row) and trend_filter_row(row):
        plot = {"hlines": [(upper_avg, "Upper (avg)")],
                "segments": [((w.index[0], H.iloc[0]), (w.index[-1], H.iloc[-1])),
                              ((w.index[0], L.iloc[0]), (w.index[-1], L.iloc[-1]))]}
        return ("Falling Wedge", "long", f"Break above wedge upper ~{upper_avg:.2f}", plot)
    return None


def detect_cup_handle(win: pd.DataFrame) -> Optional[Tuple[str, str, str, Dict[str, Any]]]:
    w = win.tail(max(MIN_CUP_BARS + 10, 50))
    if len(w) < MIN_CUP_BARS:
        return None
    hi_idx = w["High"].idxmax()
    lo_idx = w["Low"].idxmin()
    if not (hi_idx < lo_idx):
        return None
    rim_right = w.loc[lo_idx:]["High"].max()
    cup_depth = (w.loc[hi_idx, "High"] - w.loc[lo_idx, "Low"]) / max(1e-9, w.loc[hi_idx, "High"])
    if cup_depth < 0.1:
        return None
    handle = w.tail(12)
    handle_depth = (rim_right - handle["Low"].min()) / max(1e-9, rim_right)
    if handle_depth > MAX_HANDLE_DEPTH:
        return None
    t = w.index[-1]
    row = w.loc[t]
    if row["Close"] > rim_right and vol_confirm_row(row) and trend_filter_row(row):
        plot = {"hlines": [(rim_right, "Rim")], "points": [("Cup Low", lo_idx, w.loc[lo_idx, "Low"]) ]}
        return ("Cup & Handle", "long", f"Rim breakout {rim_right:.2f}", plot)
    return None


def detect_ascending_channel(win: pd.DataFrame) -> Optional[Tuple[str, str, str, Dict[str, Any]]]:
    w = win.tail(60)
    H, L = w["High"], w["Low"]
    x = np.arange(len(w))
    slope_H = np.polyfit(x, H.values, 1)[0]
    slope_L = np.polyfit(x, L.values, 1)[0]
    if not (slope_H > 0 and slope_L > 0):
        return None
    spread = (H - L)
    if spread.std() / max(1e-9, spread.mean()) > 0.25:
        return None
    t = w.index[-1]
    row = w.loc[t]
    upper = H.rolling(5).max().iloc[-1]
    lower = L.rolling(5).min().iloc[-1]
    if row["Close"] > upper and vol_confirm_row(row) and trend_filter_row(row):
        plot = {"segments": [((w.index[0], upper), (w.index[-1], upper)), ((w.index[0], lower), (w.index[-1], lower))],
                "hlines": [(upper, "Upper"), (lower, "Lower")]}
        return ("Ascending Channel", "long", f"Break above channel ~{upper:.2f}", plot)
    return None

# Priority order
DETECTORS = [
    detect_double_bottom,
    detect_ascending_triangle,
    detect_bullish_rectangle,
    detect_sym_triangle_up,
    detect_inverse_hs,
    detect_flag_pennant,
    detect_falling_wedge,
    detect_cup_handle,
    detect_ascending_channel,
]

# =========================
# SCAN LOGIC
# =========================

def scan_ticker_lastbar(ticker: str) -> Optional[Tuple[pd.Timestamp, str, str, str, Dict[str, Any], pd.DataFrame]]:
    df = load_ohlcv(ticker, START_DATE, END_DATE)
    if df.empty or len(df) < 220:
        return None
    w = df.tail(250)
    if math.isnan(w["MA200"].iloc[-1]) or math.isnan(w["MA200_slope"].iloc[-1]) or math.isnan(w["VOL_MA20"].iloc[-1]):
        return None
    for det in DETECTORS:
        out = det(w)
        if out:
            pattern, side, reason, plot_info = out
            return (w.index[-1], pattern, side, reason, plot_info, df)
    return None

# =========================
# MAIN
# =========================

def main():
    tickers = parse_tickers()
    ensure_dir(OUT_DIR)
    if SAVE_PLOTS:
        ensure_dir(PLOT_DIR)

    rows = []
    hits = 0
    saved_imgs = []

    for t in tickers:
        try:
            res = scan_ticker_lastbar(t)
        except Exception as e:
            log.warning("%s: scan error: %s", t, e)
            continue
        if res is None:
            continue
        d, pattern, side, reason, plot_info, df = res
        hits += 1
        line = f"{d.date()}  {t:<12}  {side.upper():<5}  {pattern:<24}  {reason}"
        print(line)
        rows.append({
            "date": d.date(),
            "ticker": t,
            "side": side,
            "pattern": pattern,
            "reason": reason,
        })
        if SAVE_PLOTS:
            path = save_pattern_plot(t, df, d, pattern, reason, plot_info)
            if path:
                saved_imgs.append(path)

    run_date = rows[0]["date"] if rows else pd.Timestamp.today().date()
    out_path = os.path.join(OUT_DIR, f"signals_{run_date}.csv")
    pd.DataFrame(rows).to_csv(out_path, index=False)

    if hits == 0:
        print("No breakouts today that passed trend & volume filters.")
    else:
        print(f"Saved: {out_path}")
        if SAVE_PLOTS:
            print(f"Plots: {len(saved_imgs)} files → {PLOT_DIR}")


if __name__ == "__main__":
    main()


2025-10-13  BSE.NS        LONG   Falling Wedge             Break above wedge upper ~2384.18
2025-10-13  CEATLTD.NS    LONG   Symmetrical Triangle (Up)  Upper trendline break ~3533.64
Saved: signals/signals_2025-10-13.csv
Plots: 2 files → signals/plots
